# Introduction

In this notebook, we will explore the OpenAI library. We will also acquire data in different ways, such that we can inform our queries.

The setup is not different from what we have done before.

In [1]:
%load_ext dotenv
%dotenv ../../05_src/.env
%dotenv ../../05_src/.secrets

In [2]:
import sys
sys.path.append('../../05_src/')

In [11]:
from utils.logger import get_logger
_logs = get_logger(__name__)

In [12]:
from openai import OpenAI
client = OpenAI()

2. We create an API call and store the result in the variable `response`. Notice that the call specifies the model that we want to use, as well as an input. This is a simple call, the [responses API can handle more complex calls](https://platform.openai.com/docs/api-reference/responses).

# Longer Context

Thus far, we have manually created our context and prompt. We can use any of Python text manipulation tools to create documents. 

For instance, in the code below, we will use the request library to download a book from Project Guttenberg and include it as part of the context. Some useful links are:

- Python's [request](https://pypi.org/project/requests/) library.
- Python's request library [documentation](https://requests.readthedocs.io/en/latest/).


In [5]:
import requests
file_url = 'https://www.gutenberg.org/cache/epub/223/pg223.txt'
book = requests.get(file_url)

The code below shows how to obtain the headers in the response. 

In [6]:
dict(book.headers)

{'date': 'Mon, 13 Oct 2025 20:39:29 GMT',
 'server': 'Apache',
 'last-modified': 'Wed, 01 Oct 2025 08:39:54 GMT',
 'accept-ranges': 'bytes',
 'content-length': '434868',
 'x-backend': 'gutenweb7',
 'content-type': 'text/plain; charset=utf-8'}

We can also obtain the payload using `book.text` or `book.content`.

In [20]:
print(book.text)

The Project Gutenberg eBook of The wisdom of Father Brown
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The wisdom of Father Brown

Author: G. K. Chesterton

Release date: February 1, 1995 [eBook #223]
                Most recently updated: October 9, 2016

Language: English

Credits: Produced by Martin Ward, and David Widger


*** START OF THE PROJECT GUTENBERG EBOOK THE WISDOM OF FATHER BROWN ***




Produced by Martin Ward





THE WISDOM OF FATHER BROWN

By G. K. Chesterton



To

LUCIAN OLDERSHAW



CONTENTS


     1.  The Absence of Mr Glass
     2.  The Paradise of Thieves
   

With added content, we can also write more interesting prompts. Notice the use of Python's f-strings or formatted strings. 

When using formatted strings, remember two things:

+ Formatted strings are prefixed with f:  `f'This is a formatted string: {some_variable}.`
+ You can enclose variable names in `{...}` and their values will appear in formatted strings: `f'The variable value is {variable}.`

In [8]:
prompt = f"""
    You are a literary historian. 
    Given the following context from a book, do the following:
    
    1. Identify the book's title and author.
    2. Determine how many stories are included in the book.
    3. Summarize concisely in no more than 5 paragraphs the first story of the book called "The Absence of Mr Glass".
        
    The book is the following: 
    <book>
    {book.text}
    </book>

    Provide your response in the following format:
    Title: <title>
    Author: <author>
    Number of Stories: <number_of_stories>
    Summary: <summary>
"""

In [9]:
response = client.responses.create(
    model = 'gpt-4o',
    input = prompt,
)


In [10]:
print(response.output_text)

Certainly! Here's the information you requested:

Title: The Wisdom of Father Brown  
Author: G. K. Chesterton  
Number of Stories: 12  

Summary of "The Absence of Mr Glass":

In "The Absence of Mr Glass," Father Brown arrives in Scarborough to assist an acquaintance, Father MacNab, with a delicate situation involving his parishioner's daughter, Maggie MacNab, and her secretive lodger and suitor, James Todhunter. Todhunter's strange behavior has raised suspicions that he's involved in a dangerous activity, amplified by the sealed room he often works in and a mysterious tall man spotted visiting at dusk. When Maggie believes Todhunter has been murdered, Father Brown investigates.

Upon entering the locked room, they find Todhunter bound but alive, with no sign of struggle apart from scattered playing cards and a broken glass. Father Brown astutely observes that Todhunter's restraints are cleverly self-imposed, allowing him to free himself at any time. The entire setup was a ruse to spa

# Adding a Developer Prompt

Using the OpenAI library, we can specify prompts for different roles:

+ *Developer* messages are instructions provided by the application developer, prioritized ahead of user messages. In previous iterations of the API, these were called *System* messages.
+ *User* messages are instructions provided by an end user, prioritized below developer messages.
+ *Assistant* messages are generated by the model.

In [21]:
system_prompt = "You are an anthropologist specializing in ancient fables with the vocabulary of Uncle Rogers, the TV character."

In [22]:
the_ant = """
Ants were once men and made their living by tilling the soil. But, not content with the results of their own work, they were always casting longing eyes upon the crops and fruits of their neighbours, which they stole, whenever they got the chance, and added to their own store. At last their covetousness made Jupiter so angry that he changed them into Ants. But, though their forms were changed, their nature remained the same: and so, to this day, they go about among the cornfields and gather the fruits of others’ labour, and store them up for their own use.
"""

prompt = f"""
    Please, extract the main message of this fable written by Aesop.
    The fable is the following: 
    <fable>
    {the_ant}
    </fable>
"""

In [23]:
response = client.responses.create(
    model="gpt-4o",
    instructions = system_prompt,
    input = prompt,
)

print(response.output_text)

Ah, this fable! Classic tale about greed and envy, lah! The main message is about how human nature can persist, even when appearances change. The men became ants because they couldn't stop being greedy and wanting what wasn't theirs. Jupiter turned them into ants as a punishment, but even then, they kept hoarding from others. Moral of the story: be happy with your own effort and don’t be so kiasu, okay?
